<a href="https://colab.research.google.com/github/sonalgan/DeepLearning/blob/master/02.1HPE_DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading Dataset


In [1]:
!wget -q http://lotus.kuee.kyoto-u.ac.jp/WAT/indic-multilingual/indic_wat_2021.tar.gz
!tar xzf indic_wat_2021.tar.gz
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git -q
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git -q
!mkdir data

In [1]:
punctuation=[",","।",";","?","!","—","‐",":-","'","(",")",'"']

In [2]:
languages=["hi","bn","gu","kn","mr","or","ta","te","ml","pa"]
languages.sort()

In [3]:
import sys
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()


In [4]:
import string
for char in string.punctuation:
  print(char,end=" ")
translate_table=dict((ord(char),None) for char in string.punctuation) 
global translate_table

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [12]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
factory=IndicNormalizerFactory()
from indicnlp.tokenize import sentence_tokenize

In [7]:
import csv
fields=["sentences","lang","length"]
with open('data/sentences.csv', 'w',encoding="utf-8") as outputfile:
   csvwriter=csv.writer(outputfile)
   csvwriter.writerow(fields)

Pipeline for Tokenizing the sentences while removing special char, or roman script


In [13]:
from statistics import mean,median,mode 
import os
from re import sub as resub
class LangCleaner:
  def __init__(self):
    self.lang="hi"
    self.normalizer=None

  def loader(self,lang,fp):
    self.lang=lang
    
    #fp = "finalrepo/train/pmi/en-{}/train.{}".format(lang,lang)
    with open(fp) as f:
      chunk = f.readlines()
    self.normalizer=factory.get_normalizer(lang)
    return chunk
  def cleaner(self,chunk):
    vocab=[]
    len_sentence=[]
    for line in chunk:
      line=line.rstrip("\n")
      line=self.normalizer.normalize(line)
      line= resub(r"\d+","",line)
      line=resub(r"[a-zA-Z]+","",line)
      # line=resub(r'[\u0964\u2022\u00B7]','',line)
      line=line.translate(translate_table)
      sentences=sentence_tokenize.sentence_split(line, lang=self.lang)
      

      for sentence in sentences:
        if(len(sentence)>200):
          temp=sentence[200:]
          sentence=sentence[:200]
          vocab.append(temp)
          len_sentence.append(len(temp))
        vocab.append(sentence)
        len_sentence.append(len(sentence))
    with open('data/sentences.csv', 'a',encoding="utf-8") as outputfile:
      csvwriter=csv.writer(outputfile)
      ##csvwriter.writerow(self.fields)
      for i, (x, y) in enumerate(zip(vocab, len_sentence)):
        csvwriter.writerow([x,self.lang,y])
    return len(vocab)
obj=LangCleaner()
m=[]
for lang in languages:
  for dir,subdir,files in os.walk("finalrepo/train"):
    for file in files:
      if(lang not in file):
        continue
      fp=os.path.join(dir,file)
      ans=obj.loader(lang,fp)
      len_data=obj.cleaner(ans)
  print("Length of",lang,"dataset:",len_data)
  m.append(len_data)
print("Mean: ",mean(m))
print("Median : ",median(m))

del obj
del factory
del loader




Length of bn dataset: 12691
Length of gu dataset: 16355
Length of hi dataset: 51523
Length of kn dataset: 2375
Length of ml dataset: 6479
Length of mr dataset: 23411
Length of or dataset: 36373
Length of pa dataset: 831
Length of ta dataset: 12039
Length of te dataset: 5501
Mean:  16757.8
Median :  12365.0


In [ ]:
alphabets

In [24]:

filelist = [ f for f in os.listdir("data") ]
for f in filelist:
    os.remove(os.path.join("data",f))
    


IsADirectoryError: ignored